# ETL

## Importar librerias

Se importan las librerias y desde el directorio `utils` se importa la conexion a la base de datos

In [2]:
import pandas as pd
from utils import connection as con
from etl import extract as ex
from etl import load
from etl import transform as t

## Establecer la conexion

Con sqlalchemy y las cadenas de conexion importados desde `utils` establecemos la conexion a la base de datos

In [3]:
conn_retail_db = con.connect_retail_db()
conn_dw_retail = con.connect_dw_retail()

## Metodo para importar tablas

- customers
- orders
- order_items
- products
- categories
- departments

In [4]:
df_customers = ex.extract_table('customers', conn_retail_db)
df_orders = ex.extract_table('orders', conn_retail_db)
df_order_items = ex.extract_table('order_items', conn_retail_db)
df_products = ex.extract_table('products', conn_retail_db)
df_categories = ex.extract_table('categories', conn_retail_db)
df_departments = ex.extract_table('departments', conn_retail_db)

## Cargar informacion a las tablas de OLAP

En la tabla customers de la base retail_db se debe eliminar la columna customer_password para poder cargar en la tabla de dimensiones dimension_customer. Esto debido a que no coinciden las columnas.

In [14]:
load.load_dw_retail('dimension_customer', conn_dw_retail, df_customers.drop('customer_password', axis=1))

In [15]:
load.load_dw_retail('dimension_order', conn_dw_retail, df_orders)

In [6]:
load.load_dw_retail('dimension_department', conn_dw_retail, df_departments)

In [7]:
load.load_dw_retail('dimension_category', conn_dw_retail, df_categories)

In [8]:
load.load_dw_retail('dimension_product', conn_dw_retail, df_products)

In [ ]:
load.load_dw_retail('dimension_order_item', conn_dw_retail, df_order_items)

## Transform

Transformar tipos de datos al formato correcto

In [30]:
query = """
SELECT DISTINCT
  DATE_FORMAT(order_date, '%%Y%%m%%d') AS time_id,
  YEAR(order_date) AS year,
  QUARTER(order_date) AS quarter,
  MONTH(order_date) AS month,
  DAY(order_date) AS day,
  DAYOFWEEK(order_date) AS day_of_week,
  DAYOFYEAR(order_date) AS day_of_year,
  WEEK(order_date) AS week_of_year,
  CASE 
    WHEN DAYOFWEEK(order_date) IN (1,7) THEN TRUE 
    ELSE FALSE 
  END AS is_weekend
FROM retail_db.orders
"""

df_dimension_time = t.transform_time_from_order(conn_retail_db, query)

## Load transformed data

Cargar la data que ya fue transformada en la tabla correspondiente

In [24]:
load.load_dw_retail('dimension_time', conn_dw_retail, df_dimension_time)

In [83]:
query = """
SELECT
  o.order_id,
  o.order_date,
  o.order_customer_id,
  o.order_status,
  SUM(oi.order_item_quantity) AS total_items,
  SUM(oi.order_item_subtotal) AS total_amount,
  DATE_FORMAT(o.order_date, '%%Y%%m%%d') as time_id
FROM retail_db.orders o
INNER JOIN retail_db.order_items oi ON o.order_id = oi.order_item_order_id
INNER JOIN dw_retail.dimension_time dt ON DATE_FORMAT(o.order_date, '%%Y%%m%%d') = dt.time_id
GROUP BY o.order_id
"""

df_fact_orders = t.transform_time_from_order(conn_dw_retail, query)

In [84]:
load.load_dw_retail('fact_orders', conn_dw_retail, df_fact_orders)
conn_dw_retail.commit()

In [4]:
conn_dw_retail.close()